# Segmenting and Clustering Neighborhoods in Toronto Part ( Week 3)

# Part 1

<H2><b>Tasks to Complete :</b></H2>
<ol>
 <li>
     <B>Task 1 : </B>Dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 </li>
  <li>   
 <B>Task 2 : </B>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 </li>
 <li>   
<B>Task 3 : </B> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed  twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
      </li>
   <li>    
<B>Task 4 : </B>If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
       </li> 
       <li> 
<B>Task 5 : </B>    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
    </li>    

    
</ol>    
    

<H2><b>Tasks 1::</b></H2>
Dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Import Packages

In [1]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
from bs4 import BeautifulSoup 
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import re

In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

<p><b>Find All Tables</b></p>

In [4]:
table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res=[]
data = []
for tr in table_rows:
    td = tr.find_all("td")
    for tr in td:
        # Get the Text  , Now i need to Break it
          # Separate the  cell to  ( Potcode , Borough  , Neighbourhood)
            
       
      
        row = tr.text 
        
        #Get the Postcode
       
        postCode=row[:4]
        #print(postCode)
        #data.append(postCode)
        
        #Get Remaining String
        
        remains = row[4:]
        
        
        
        # Get Borough &  Neighbourhood
        borough= remains 
        
        new_borough=borough.split('(')[0]
        #print(new_borough)
        #data.append(new_borough)
        
        #new_neigh=re.sub(r"(.*?)\s?\(.*?\)", r"\1", borough).lower()
        #new_neigh = re.findall(r'\(.*?\)', borough)
        tt = re.search(r'\((.+)\)', borough) 
        if tt: 
            new_neigh = tt.group(0)
            #print(new_neigh)
            data.append(new_neigh)
        else:
            new_neigh = ""
            #data.append(new_neigh)
        #res.append(data)
        res.append([postCode,new_borough,new_neigh])
            

finaldata= pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
      
              
            

    

In [5]:
display(finaldata)

,PostalCode,Borough,Neighborhood
0,\nM1A,Not assigned\n\n,
1,\nM2A,Not assigned\n\n,
2,\nM3A,North York,(Parkwoods)
3,\nM4A,North York,(Victoria Village)
4,\nM5A,Downtown Toronto,(Regent Park / Harbourfront)
...,...,...,...
175,\nM5Z,Not assigned\n\n,
176,\nM6Z,Not assigned\n\n,
177,\nM7Z,Not assigned\n\n,
178,\nM8Z,Etobicoke,(Mimico NW / The Queensway West / South of Blo...


<p><b> Clean Data (Remove all"\n" in the Post Code Column "\n\n" in Borough Column Bracket in Neighbourhood Column </b> </p>

In [6]:
finaldata['PostalCode']=finaldata['PostalCode'].str.lstrip('\n')
finaldata['Borough']=finaldata['Borough'].str.rstrip('\n')

spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    finaldata['Neighborhood'] = finaldata['Neighborhood'].str.replace(char, ' ')



display(finaldata)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Blo...


<h2><b>Tasks 2:</b></h2>
 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

I will drop the Whole Row  when detect Borough Column  = "Not assigned "

In [9]:
finaldata.drop(finaldata.loc[finaldata['Borough']=="Not assigned"].index, inplace=True)

Check the DataFrame After Drop

In [10]:
display(finaldata)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
169,M8Y,Etobicoke,Old Mill South / King s Mill Park / Sunnylea ...


<h2><b>Tasks 3:</b></h2>
 More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

Replace the Character "/" with ","

In [12]:
finaldata['Neighborhood'] = finaldata['Neighborhood'].str.replace('/',',')

Check the Dataframe After Replace

In [13]:
display(finaldata)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
169,M8Y,Etobicoke,"Old Mill South , King s Mill Park , Sunnylea ..."


<h2><b>Tasks 4:<b></H2>
 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [17]:
finaldata.fillna(finaldata['Neighborhood'], inplace=True)

In [18]:
display(finaldata)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
169,M8Y,Etobicoke,"Old Mill South , King s Mill Park , Sunnylea ..."


<h2><b>Tasks 5:</b> </h2>
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [19]:
finaldata.shape

(103, 3)

# Part2 

<h2><b>Tasks to Complete :</b></h2>
<ol>
    <li>
        <b>Task A:  </b> Get the latitude and the longitude coordinates of each neighborhood. 
    </li>
    <li>
       <b>Task B:  </b> Create a Data Frame which Shows the Following Column (Postal Code , Borough , Latitude , Longtitude)  
    </li>
    
</ol>

<h2>Task A : Get the latitude and the longitude coordinates of each neighborhood. </h2>

In [23]:
geo_coordinate = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
geo_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2>Task B : </h2>
Create a Data Frame which Shows the Following Column (Postal Code , Borough , Latitude , Longtitude)  

Merge DataFrame  ( finaldata with geo_coordinate ) where Attribute   finaldata (PostalCode) Equal geo_coordinate (Postal Code)

In [24]:
df_With_PostCode = pd.merge(finaldata, geo_coordinate, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_With_PostCode.drop("Postal Code", axis=1, inplace=True)
df_With_PostCode.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

<ol>
    <li>
        <b>Task I:  </b> To add enough Markdown cells to explain what you decided to do and to report any observations you make.  
    </li>
    <li>
       <b>Task II:  </b> To generate maps to visualize your neighborhoods and how they cluster together.  
    </li>
    
</ol>

<H2> I will like to Work with Down Town Toronto As  an Example</h2>

<b>Scenario</b>
<ol>
<li>Let Say I staying in <b> The Novotel Down Town Toronto</b> located at Down Town Toronto</li>
<li>I Would like to look For a Good <b>"Japanese Restaurant"</b> to Dine in</li>

    
</ol>


First I take a Glimpse Of all the Location  (Postcode , Neighbourhood  , Latitude, Longtitude ) Located in Down Town Toronto

In [26]:
df_With_PostCode[df_With_PostCode.Borough.str.contains('Downtown Toronto',case=False)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,Garden District Ryerson,43.657162,-79.378937
15,M5C,Downtown Toronto,St James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East , Union Station , Toronto I...",43.640816,-79.381752
42,M5K,Downtown Toronto,"Toronto Dominion Centre , Design Exchange",43.647177,-79.381576
48,M5L,Downtown Toronto,"Commerce Court , Victoria Hotel",43.648198,-79.379817


# Connecting To Foursquare API

Credential To access Four Square

In [31]:
CLIENT_ID = 'T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC' # your Foursquare ID
CLIENT_SECRET = 'HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX' # your Foursquare Secret
ACCESS_TOKEN = 'GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC
CLIENT_SECRET:HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX


<h2>Get the GEO Location of the Hotel <b><i style="color:green">"The Novotel Down Town Toronto</i>"</b> where I stay</h2>

In [34]:
address = '45 The Esplanade, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.646654 -79.37411907725917


<h2>Now Search For Japanese Reaturant Around The Hotel where I stay</h2>

In [39]:
search_query = 'japanese'
radius = 500
print(search_query + ' ...OK!')

japanese ...OK!


<b>Define the corresponding URL ( Client_id / CLient_secret / Access Token ) Already being defined on top</b>

In [40]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC&client_secret=HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX&ll=43.646654,-79.37411907725917&oauth_token=GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC&v=20180604&query=japanese&radius=500&limit=30'

<b>Send the GET Request and examine the results</b>

In [41]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607698f433050e640ae09ce5'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ae73054f964a5203ca921e3',
    'name': 'Ki Modern Japanese + Bar',
    'location': {'address': '181 Bay St',
     'crossStreet': 'at Wellington St. W',
     'lat': 43.647223,
     'lng': -79.3793738,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.647223,
       'lng': -79.3793738}],
     'distance': 427,
     'postalCode': 'M5J 2T3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['181 Bay St (at Wellington St. W)',
      'Toronto ON M5J 2T3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d111941735',
      'name': 'Japanese Restaurant',
      'pluralName': 'Japanese Restaurants',
      'shortName': 'Japanese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
       'suffix': '.png'},
    

<b> Get Derived JSON Information  and transform it into a pandas dataframe </b>

In [42]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(20)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ae73054f964a5203ca921e3,Ki Modern Japanese + Bar,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1618385140,False,181 Bay St,at Wellington St. W,43.647223,-79.379374,"[{'label': 'display', 'lat': 43.647223, 'lng':...",427,M5J 2T3,CA,Toronto,ON,Canada,"[181 Bay St (at Wellington St. W), Toronto ON ...",NaN
1,4af63562f964a5203d0222e3,Bikkuri Japanese,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1618385140,False,36 King St. E,at Victoria,43.649564,-79.376328,"[{'label': 'display', 'lat': 43.649564308142, ...",369,NaN,CA,Toronto,ON,Canada,"[36 King St. E (at Victoria), Toronto ON, Canada]",NaN
2,574ad72238fa943556d93b8e,Gyu-Kaku Japanese BBQ,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1618385140,False,81 Church St,at Adelaide St E,43.651422,-79.375047,"[{'label': 'display', 'lat': 43.65142227549791...",536,M5C 2G2,CA,Toronto,ON,Canada,"[81 Church St (at Adelaide St E), Toronto ON M...",NaN
3,4b999927f964a5207c8635e3,NAMI,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1618385140,False,55 Adelaide Street East,at Church street,43.650853,-79.375887,"[{'label': 'display', 'lat': 43.65085336178512...",488,M5C 1K6,CA,Toronto,ON,Canada,"[55 Adelaide Street East (at Church street), T...",NaN
4,4beb2c0ba9900f475b7a1740,Ninki Sushi,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1618385140,False,40 King St W,at Bay St,43.649812,-79.379518,"[{'label': 'display', 'lat': 43.64981217424400...",559,M5H 1A1,CA,Toronto,ON,Canada,"[40 King St W (at Bay St), Toronto ON M5H 1A1,...",Financial District


<b>Define information of interest and filter dataframe</b>

In [44]:
# keep only columns that include venue name
# anything that is associated with location
#Add   ['name'] + ['categories']  Column + Filter All Column only remain those with location.xx + add  the 'id' column


filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']

# Assign the Entire':'-> Means Entire Column of  filtered_column into a new dataframe called 'dataframe_filtered'
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Ki Modern Japanese + Bar,Japanese Restaurant,181 Bay St,at Wellington St. W,43.647223,-79.379374,"[{'label': 'display', 'lat': 43.647223, 'lng':...",427,M5J 2T3,CA,Toronto,ON,Canada,"[181 Bay St (at Wellington St. W), Toronto ON ...",NaN,4ae73054f964a5203ca921e3
1,Bikkuri Japanese,Sushi Restaurant,36 King St. E,at Victoria,43.649564,-79.376328,"[{'label': 'display', 'lat': 43.649564308142, ...",369,NaN,CA,Toronto,ON,Canada,"[36 King St. E (at Victoria), Toronto ON, Canada]",NaN,4af63562f964a5203d0222e3
2,Gyu-Kaku Japanese BBQ,Japanese Restaurant,81 Church St,at Adelaide St E,43.651422,-79.375047,"[{'label': 'display', 'lat': 43.65142227549791...",536,M5C 2G2,CA,Toronto,ON,Canada,"[81 Church St (at Adelaide St E), Toronto ON M...",NaN,574ad72238fa943556d93b8e
3,NAMI,Japanese Restaurant,55 Adelaide Street East,at Church street,43.650853,-79.375887,"[{'label': 'display', 'lat': 43.65085336178512...",488,M5C 1K6,CA,Toronto,ON,Canada,"[55 Adelaide Street East (at Church street), T...",NaN,4b999927f964a5207c8635e3
4,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,43.649812,-79.379518,"[{'label': 'display', 'lat': 43.64981217424400...",559,M5H 1A1,CA,Toronto,ON,Canada,"[40 King St W (at Bay St), Toronto ON M5H 1A1,...",Financial District,4beb2c0ba9900f475b7a1740


 <b> visualize nearby Japanese restaurants </b>

In [46]:
dataframe_filtered.name

0    Ki Modern Japanese + Bar
1            Bikkuri Japanese
2       Gyu-Kaku Japanese BBQ
3                        NAMI
4                 Ninki Sushi
Name: name, dtype: object

<b>There Are a Total of 5 Restaurant Near By </b>

<h2>Generate Map Around "The Novotel Down Town Toronto" Hotel & Japanese Restaurant NearBy the Hotel</h2>

<ul>
<li>
Red Marker -> The  Novotel Down Town Location

</li>
<li>
</li>
Blue Marker -> The  Japanese Restaurant Nearby Available

</ul>

In [51]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

# 
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='The Novotel Down Town Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Now Explore All Japanese Restaurant Rating

<h2> First Restaurant Ki Modern Japanese + Bar </h2>

In [53]:
venue_id = '4ae73054f964a5203ca921e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4ae73054f964a5203ca921e3?client_id=T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC&client_secret=HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX&oauth_token=GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC&v=20180604'

<b>Send GET request for result</b>

In [54]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ae73054f964a5203ca921e3',
 'name': 'Ki Modern Japanese + Bar',
 'contact': {'phone': '+14163085888',
  'formattedPhone': '+1 416-308-5888',
  'facebook': '115585431834030',
  'facebookUsername': 'kijapanesetoronto',
  'facebookName': 'Ki Modern Japanese + Bar'},
 'location': {'address': '181 Bay St',
  'crossStreet': 'at Wellington St. W',
  'lat': 43.647223,
  'lng': -79.3793738,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.647223,
    'lng': -79.3793738}],
  'postalCode': 'M5J 2T3',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['181 Bay St (at Wellington St. W)',
   'Toronto ON M5J 2T3',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/ki-modern-japanese--bar/4ae73054f964a5203ca921e3',
 'categories': [{'id': '4bf58dd8d48988d111941735',
   'name': 'Japanese Restaurant',
   'pluralName': 'Japanese Restaurants',
   'shortName': 'Japanese',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/j

<b>Get the venue's overall rating</b>

In [56]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


<h2> Second Restaurant Bikkuri Japanese Lets Check the Rating</h2>

In [61]:
venue_id_2 = '4af63562f964a5203d0222e3' 
url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_2, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url2

'https://api.foursquare.com/v2/venues/4af63562f964a5203d0222e3?client_id=T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC&client_secret=HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX&oauth_token=GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC&v=20180604'

In [63]:
result2 = requests.get(url2).json()
print(result2['response']['venue'].keys())
result2['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4af63562f964a5203d0222e3',
 'name': 'Bikkuri Japanese',
 'contact': {'phone': '+14163690330', 'formattedPhone': '+1 416-369-0330'},
 'location': {'address': '36 King St. E',
  'crossStreet': 'at Victoria',
  'lat': 43.649564308142,
  'lng': -79.37632789169632,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.649564308142,
    'lng': -79.37632789169632}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['36 King St. E (at Victoria)', 'Toronto ON', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/bikkuri-japanese/4af63562f964a5203d0222e3',
 'categories': [{'id': '4bf58dd8d48988d1d2941735',
   'name': 'Sushi Restaurant',
   'pluralName': 'Sushi Restaurants',
   'shortName': 'Sushi',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d111941735',
   'name': 'Japanese Restaurant',
   'pluralName': 'Japanese Restaurants',
   'short

<h2>Get Overall Rating</h2>

In [64]:
try:
    print(result2['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.1


<h2>Third Japanese Restaurant"  Gyu-Kaku Japanese BBQ"</h2>

In [66]:
venue_id_3 = '4af63562f964a5203d0222e3' 
url3 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_3, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url3

'https://api.foursquare.com/v2/venues/4af63562f964a5203d0222e3?client_id=T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC&client_secret=HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX&oauth_token=GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC&v=20180604'

In [68]:
result3 = requests.get(url3).json()
print(result3['response']['venue'].keys())
result3['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4af63562f964a5203d0222e3',
 'name': 'Bikkuri Japanese',
 'contact': {'phone': '+14163690330', 'formattedPhone': '+1 416-369-0330'},
 'location': {'address': '36 King St. E',
  'crossStreet': 'at Victoria',
  'lat': 43.649564308142,
  'lng': -79.37632789169632,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.649564308142,
    'lng': -79.37632789169632}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['36 King St. E (at Victoria)', 'Toronto ON', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/bikkuri-japanese/4af63562f964a5203d0222e3',
 'categories': [{'id': '4bf58dd8d48988d1d2941735',
   'name': 'Sushi Restaurant',
   'pluralName': 'Sushi Restaurants',
   'shortName': 'Sushi',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d111941735',
   'name': 'Japanese Restaurant',
   'pluralName': 'Japanese Restaurants',
   'short

<h2>Get Overall Raing</h2>

In [70]:
try:
    print(result3['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.1


<h2>Fouth Restaurant NAMI</h2>

In [72]:
venue_id_4 = '4b999927f964a5207c8635e3' 
url4 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_4, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url4

'https://api.foursquare.com/v2/venues/4b999927f964a5207c8635e3?client_id=T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC&client_secret=HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX&oauth_token=GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC&v=20180604'

In [73]:
result4 = requests.get(url4).json()
print(result4['response']['venue'].keys())
result4['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4b999927f964a5207c8635e3',
 'name': 'NAMI',
 'contact': {'phone': '+14163627373', 'formattedPhone': '+1 416-362-7373'},
 'location': {'address': '55 Adelaide Street East',
  'crossStreet': 'at Church street',
  'lat': 43.650853361785124,
  'lng': -79.37588746641697,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.650853361785124,
    'lng': -79.37588746641697}],
  'postalCode': 'M5C 1K6',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['55 Adelaide Street East (at Church street)',
   'Toronto ON M5C 1K6',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/nami/4b999927f964a5207c8635e3',
 'categories': [{'id': '4bf58dd8d48988d111941735',
   'name': 'Japanese Restaurant',
   'pluralName': 'Japanese Restaurants',
   'shortName': 'Japanese',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d1d2941735',
   'name': 'Sushi R

<h2>Get Overall Rating</h2>

In [75]:
try:
    print(result4['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.9


<H2>Fifth Restaurant Ninki Sushi</H2>

In [77]:
venue_id_5 = '4beb2c0ba9900f475b7a1740' 
url5 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id_5, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url5

'https://api.foursquare.com/v2/venues/4beb2c0ba9900f475b7a1740?client_id=T15SDXVGR055HT5NGX0DTFW5CCDLVAW4GGUKJU1QTWOD4XPC&client_secret=HETI5PMYVBGXDNW2XG2KRIPD3EWA0MUNFQ3ZWUR1PZUGISRX&oauth_token=GJ1BUBLBS02MW1PSGD0PJKTVDROECJCAPH2SAHL3LRXZSHBC&v=20180604'

In [78]:
result5 = requests.get(url5).json()
print(result5['response']['venue'].keys())
result5['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'id': '4beb2c0ba9900f475b7a1740',
 'name': 'Ninki Sushi',
 'contact': {'phone': '+14163687333',
  'formattedPhone': '+1 416-368-7333',
  'facebook': '523844677635561',
  'facebookUsername': 'NinkiIzakaya',
  'facebookName': 'NINKI Japanese Cuisine'},
 'location': {'address': '40 King St W',
  'crossStreet': 'at Bay St',
  'lat': 43.649812174244005,
  'lng': -79.37951837924346,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.649812174244005,
    'lng': -79.37951837924346}],
  'postalCode': 'M5H 1A1',
  'cc': 'CA',
  'neighborhood': 'Financial District',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['40 King St W (at Bay St)',
   'Toronto ON M5H 1A1',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/ninki-sushi/4beb2c0ba9900f475b7a1740',
 'categories': [{'id': '4bf58dd8d48988d111941735',
   'name': 'Japanese Restaurant',
   'pluralName': 'Japanese Restaurants',
   'shortName': 'Japanese',
   'icon': {'prefix': 'https://ss3.4sqi.net/

In [79]:
try:
    print(result5['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.5


# Summary ( Which Japanese Retaurant To Go)

<ul>
<li>
 Ki Modern Japanese + Bar -> <b>Rating 7.7🌟</b>
</li>
<li>
 Bikkuri Japanese Restaurant -> <b>Rating 5.1🌟</b>
</li>
<li>
Gyu-Kaku Japanese BBQ Restaurant -> <b>Rating 5.1🌟</b>
</li>
<li>
NAMI Japanese  Restaurant -> <b>Rating 7.9🌟</b>
</li>
<li>
Ninki Sushi Japanese  Restaurant -> <b>Rating 5.5🌟</b>
</li>

</ul>

<h2>I have Made Up My Decision to go for  "NAMI Japanese Restaurant"</h2>